In [1]:
import requests, pandas as pd, numpy as np
import time
from bs4 import BeautifulSoup
from random import sample
import os

In [2]:
from webdriver_manager.firefox import GeckoDriverManager
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
os.chdir('Similarity Data/Regular Season')
os.getcwd()

'C:\\Users\\spens\\Basketball\\NBA Tracking\\Similarity Data\\Regular Season'

In [4]:
df = pd.read_csv('track_shooting-efficiency_reg_szn_similarity.csv')
df.drop(columns=["Unnamed: 0"], inplace=True)

In [5]:
df.index = list(df.columns)
df = df.dropna(axis=0, thresh=len(df)*0.3)
df = df.dropna(axis=1, thresh=len(df)*0.3)
df

,Jimmy Butler,James Harden,Kyle Lowry,LeBron James,John Wall,Paul George,Damian Lillard,Andrew Wiggins,Anthony Davis,Devin Booker,...,Shamorie Ponds,Erik Murphy,Jared Harper,Ike Anigbogu,Reggie Hearn,Jacob Pullen,Coty Clarke,Vincent Hunter,Trevon Duval,Sim Bhullar
Jimmy Butler,1.000000,0.993206,0.995996,0.995749,0.981205,0.996529,0.984689,0.997552,0.997452,0.992600,...,0.593178,0.598709,0.513989,0.746965,0.464746,0.577253,0.466192,0.781639,0.575013,0.513670
James Harden,0.993206,1.000000,0.993293,0.990004,0.983045,0.995693,0.990739,0.991042,0.989105,0.993664,...,0.628617,0.527028,0.538622,0.750143,0.492476,0.595621,0.494392,0.779788,0.608415,0.524082
Kyle Lowry,0.995996,0.993293,1.000000,0.995372,0.986933,0.996253,0.987114,0.997104,0.991138,0.995152,...,0.605656,0.554612,0.509630,0.791237,0.490645,0.568375,0.491976,0.816398,0.594776,0.518627
LeBron James,0.995749,0.990004,0.995372,1.000000,0.981332,0.990240,0.982997,0.998379,0.992629,0.992469,...,0.566438,0.586477,0.501889,0.778617,0.432351,0.536144,0.433901,0.798570,0.543700,0.498739
John Wall,0.981205,0.983045,0.986933,0.981332,1.000000,0.987089,0.957303,0.977673,0.975474,0.972595,...,0.645214,0.540602,0.553883,0.754913,0.499443,0.613560,0.500896,0.787936,0.620846,0.413371
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Jacob Pullen,0.577253,0.595621,0.568375,0.536144,0.613560,0.589569,0.555791,0.540494,0.553568,0.543029,...,0.570351,0.098131,0.745332,0.108270,0.200018,1.000000,0.200016,0.118293,0.438250,0.235718
Coty Clarke,0.466192,0.494392,0.491976,0.433901,0.500896,0.516358,0.474014,0.457403,0.475863,0.473710,...,0.773150,0.176635,0.268417,0.545912,0.999933,0.200016,1.000000,0.685902,0.932232,0.424271
Vincent Hunter,0.781639,0.779788,0.816398,0.798570,0.787936,0.794704,0.798994,0.807495,0.784730,0.815196,...,0.532155,0.387008,0.158743,0.969299,0.685934,0.118293,0.685902,1.000000,0.636106,0.459979
Trevon Duval,0.575013,0.608415,0.594776,0.543700,0.620846,0.619013,0.574078,0.562194,0.573928,0.565445,...,0.946843,0.175889,0.588009,0.508781,0.932258,0.438250,0.932232,0.636106,1.000000,0.422500


In [6]:
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())



====== WebDriver manager ======
Current firefox version is 102.0
Get LATEST geckodriver version for 102.0 firefox
Driver [C:\Users\spens\.wdm\drivers\geckodriver\win64\v0.31.0\geckodriver.exe] found in cache
C:\Users\spens\AppData\Local\Temp/ipykernel_17888/3328504527.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())


In [7]:
def find_highlights(fav_name_lst, url):
    fav_fname = fav_name_lst[0]
    fav_lname = fav_name_lst[1]
    player = str(fav_fname) + " " + str(fav_lname)
    url = url.format(fav_fname, fav_lname)
    #print(url)
    driver.get(url)
    time.sleep(3.5)
    elems = driver.find_elements(By.XPATH,'//*[@id="video-title"]')
    links = [elem.get_attribute('href') for elem in elems]
    #print(len(links))
    if len(links) > 1:
        #print(links)
        links = links[:7]
        res = sample(links, 1)
        res = res[0]
        print(str(player.title()) + " highlights")
        print("\t"+str(res))

In [8]:
def get_player_recc():
    player = str(input("Enter the name of a player like to watch: "))
    url = 'https://www.youtube.com/results?search_query=nba+{}+{}+highlights&sp=CAASAhAB'
    #print(player)
    print("Your player is: " + str(player.title()))
    fav_name_lst = player.split(' ')
    find_highlights(fav_name_lst, url)
    sim_lst = list(df[str(player)].sort_values(ascending=False).head(15).index)
    p = sim_lst.pop(0)
    #print(p == player)
    sim_players = sample(sim_lst,1)
    print("Here's a player you might enjoy watching then... ")
    p = sim_players[0]
    #print("\t> " + str(p.title()))
    name_lst = p.split(' ')
    find_highlights(name_lst, url)
    driver.quit()

In [9]:
get_player_recc()

Enter the name of a player like to watch: Nikola Jokic
Your player is: Nikola Jokic
Nikola Jokic highlights
	https://www.youtube.com/watch?v=FGmps-x1TzQ
Here's a player you might enjoy watching then... 
Blake Griffin highlights
	https://www.youtube.com/watch?v=hoarqsrBULo
